(ex_system_rev)=

# Using the expression system effectively

## Long sums and products

heyoka.py internally represents sums and products as multivariate functions. E.g., the expression $x+y+z$, is represented as a single function with 3 arguments $x$, $y$ and $z$, rather than two nested binary additions, i.e., $\left( x + y \right) + z$. While such a representation has the advantage of being well-suited for the implementation of several automatic simplifications, it results in quadratic complexity when iteratively building long sums and products via repeated additions and multiplications. For instance, consider the following code for the construction of the expression $x_0 + x_1 + \ldots + x_9$:

In [1]:
import heyoka as hy

long_sum = hy.expression(0.)

# BAD: quadratic complexity.
for i in range(10):
    long_sum += hy.expression(f"x_{i}")
    
long_sum

(x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9)

While this code is correct, every time the addition operator is invoked in the loop a new copy of the arguments in ``long_sum`` is created, resulting in overall quadratic complexity. This behaviour can be avoided by preparing a list of terms for the summation and then inovking the ``heyoka.sum()`` function:

In [2]:
terms = [hy.expression(f"x_{i}") for i in range(10)]
# GOOD: linear complexity.
long_sum = hy.sum(terms)

long_sum

(x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9)

Similarly for products:

In [3]:
terms = [hy.expression(f"x_{i}") for i in range(10)]
# GOOD: linear complexity.
long_prod = hy.prod(terms)

long_prod

(x_0 * x_1 * x_2 * x_3 * x_4 * x_5 * x_6 * x_7 * x_8 * x_9)

## Automatic simplifications

heyoka.py's expression system implements several simplifications/normalisations which are automatically applied when an expression is created. Constants, for instance, are automatically folded:

In [4]:
hy.expression(1.) + hy.expression(2.)

3.0000000000000000

In [5]:
hy.cos(hy.expression(.5))

0.87758256189037276

Nested sums and products are automatically flattened:

In [6]:
x, y, z = hy.make_vars("x", "y", "z")

((1. + x) + y) + z

(1.0000000000000000 + x + y + z)

In [7]:
((2. * x) * y) * z

(2.0000000000000000 * x * y * z)

Terms in sums and products are sorted according to a canonical order:

In [8]:
y + x + 1.

(1.0000000000000000 + x + y)

Sums of products with numerical coefficients are automatically gathered:

In [9]:
2.*x + 3.*x

(5.0000000000000000 * x)

Similarly, products of exponentiations with numerical exponents are also gathered:

In [10]:
x**2. * x**3.

x**5.0000000000000000

The product of a numerical coefficient and a sum is automatically expanded:

In [11]:
2. * (x + y + z)

((2.0000000000000000 * x) + (2.0000000000000000 * y) + (2.0000000000000000 * z))

Nested exponentiations are flattened:

In [12]:
(x**2.)**3.

x**6.0000000000000000

Exponentiations of products with integral exponents are expanded:

In [13]:
(x*y)**3.

(x**3.0000000000000000 * y**3.0000000000000000)

These simplifcations are almost always automatically applied. One notable exception is the substitution function ``subs()``, which, by default, does not apply any automatic simplification:

In [14]:
hy.subs(x + y, {x : y})

(y + y)

As you can see, $y+y$ has not been simplified to $2y$. We can explicitly request the application of the automatic simplifications rules either by invoking the ``normalise()`` function:

In [15]:
hy.normalise(hy.subs(x + y, {x : y}))

(2.0000000000000000 * y)

Or, equivalently, by passing the ``normalise=True`` flag when invoking ``subs()``:

In [16]:
hy.subs(x + y, {x : y}, normalise=True)

(2.0000000000000000 * y)

(ex_system_prev_simpl)=

## Preventing automatic simplifications

It can sometimes be desirable to prevent the simplifications automatically applied by the expression system. Consider, for instance, the vector-valued function

$$
\boldsymbol{f}\left(x, y, z\right) = \left(x + y + 1, x + y + z\right).
$$

We can implement in heyoka.py a [compiled function](<./compiled_functions.ipynb>) for the fast evaluation of $\boldsymbol{f}\left(x, y, z\right)$:

In [17]:
f_cf = hy.cfunc([x+y+1., x+y+z], [x, y, z])

[2024-02-12 07:51:33.090] [heyoka] [info] heyoka logger initialised


Let us take a look at the decomposition of $\boldsymbol{f}\left(x, y, z\right)$:

In [18]:
f_cf.dc

[x, y, z, (1.0000000000000000 + u_0 + u_1), (u_0 + u_1 + u_2), u_3, u_4]

As usual, the first three elements are the inputs of the function $\left(x, y, z\right)$, while the last two elements represent the outputs of the function (i.e., $u_3$ indicates that the first output is the expression at index 3 in the decomposition, while $u_4$ indicates that the second output is the expression at index 4 in the decomposition). The two elements in the middle of the decomposition are the components of $\boldsymbol{f}\left(x, y, z\right)$, which happen to be already elementary subexpressions (and which are thus not decomposed any further).

A visual inspection of $\boldsymbol{f}\left(x, y, z\right)$ immediately shows how the expression $x+y$ appears in both components. heyoka.py's compiled functions are usually able to identify and remove repeated subexpressions from the decomposition in order to avoid repeated (and redundant) evaluations of the same subexpressions. However, in this specific case $x+y$ does not show up in the decomposition as a standalone subexpression, because it is part of the ternary sums $x + y + 1$ and $x + y + z$. One may be tempted to use brackets to isolate $x+y$ from the other terms of the sums, but this approach will not work due to the expression system automatically flattening nested sums (and essentially removing the extra brackets):

In [19]:
(x + y) + 1.

(1.0000000000000000 + x + y)

For these situations, heyoka.py provides a function called ``fix()`` that acts as a barrier to automatic simplifications. Let us try it:

In [20]:
hy.fix(x + y) + 1.

(1.0000000000000000 + {(x + y)})

The curly brackets indicate that $x+y$ has been fixed. Because $x+y$ is now wrapped in a ``fix()`` function, we are not dealing any more with a nested summation, and no automatic flattening is applied by heyoka.py.

Let us try to compile a new version of $\boldsymbol{f}\left(x, y, z\right)$ in which $x+y$ is fixed in both components:

In [21]:
f_cf_fix = hy.cfunc([hy.fix(x+y)+1., hy.fix(x+y)+z],
                        [x, y, z])

And let us take a look at the decomposition:

In [22]:
f_cf_fix.dc

[x, y, z, (u_0 + u_1), (1.0000000000000000 + u_3), (u_2 + u_3), u_4, u_5]

We can now see how in the decomposition $x + y$ is computed only once (as $u_0 + u_1$), and the result is then re-used in the computation of both outputs. As a result, the total number of floating-point operations necessary to evaluate $\boldsymbol{f}\left(x, y, z\right)$ has been reduced from 4 (in ``f_cf``) to 3 (in ``f_cf_fix``).

As another example, consider the scalar function

$$
g\left(x, y, z\right) = 2\left(x+y+z\right).
$$

If we implement this function in heyoka.py, the product is immediately expanded due to the automatic simplification rules:

In [23]:
2.0 * (x + y + z)

((2.0000000000000000 * x) + (2.0000000000000000 * y) + (2.0000000000000000 * z))

Let us compile $g\left(x, y, z\right)$ and let us take a look at the decomposition:

In [24]:
g_cf = hy.cfunc([2.0 * (x + y + z)], [x, y, z])
g_cf.dc

[x,
 y,
 z,
 (2.0000000000000000 * u_0),
 (2.0000000000000000 * u_1),
 (2.0000000000000000 * u_2),
 (u_3 + u_4 + u_5),
 u_6]

Clearly, this is suboptimal as 3 multiplications are needed for the evaluation instead of only 1. Let us try to ``fix()`` $\left( x + y + z \right)$ in order to prevent the automatic expansion:

In [25]:
g_cf_fix = hy.cfunc([2.0 * hy.fix(x + y + z)],
                        [x, y, z])
g_cf_fix.dc

[x, y, z, (u_0 + u_1 + u_2), (2.0000000000000000 * u_3), u_4]

Indeed, the expansion has been prevented and as a result the total number of operations necessary to evaluate $g\left(x, y, z\right)$ has been reduced from 5 to 3.

In addition to ``fix()``, the closely-related ``fix_nn()`` function is also available. ``fix_nn()`` will fix an expression only if it is not a number:

In [26]:
# fix() prevents constant folding.
hy.cos(hy.fix(hy.expression(.5)))

cos({0.50000000000000000})

In [27]:
# fix_nn() allows constant folding.
hy.cos(hy.fix_nn(hy.expression(.5)))

0.87758256189037276

The ``unfix()`` function can be used to remove all instances of ``fix()`` within an expression:

In [28]:
ex = 2.0 * hy.fix(x + y + z)

# Unfix ex.
ex_unfix = hy.unfix(ex)
ex_unfix

(2.0000000000000000 * (x + y + z))

Note how ``unfix()`` only removes the instances of ``fix()`` within the expression tree, but it does not re-apply the automatic simplifications. Automatic simplifications can be applied via the ``normalise()`` function:

In [29]:
hy.normalise(ex_unfix)

((2.0000000000000000 * x) + (2.0000000000000000 * y) + (2.0000000000000000 * z))